In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from scipy.special import expit
digits = datasets.fetch_mldata("MNIST Original")


In [12]:
digits.data = np.array(digits.data, 'float32') 
digits.target = np.array(digits.target, 'int16')

#digits.data = digits.data.reshape((len(features),28, 28))
minVal = np.amin(digits.data)

digits.data = (digits.data - np.amin(digits.data))/(np.amax(digits.data) - minVal)

permutation = np.random.permutation(len(digits.data))

digits.data = digits.data[permutation]
digits.target = digits.target[permutation]

In [13]:
class neuralNetwork(object):
    def __init__(self):
        self.inputLayerS=784
        self.outputLayerS=10
        self.hiddenLayer1S=500
        self.hiddenLayer2S=500
        
        np.random.seed(1)
        
        self.w1 = np.random.randn(self.inputLayerS, self.hiddenLayer1S)
        self.w2 = np.random.randn(self.hiddenLayer1S, self.hiddenLayer2S)
        self.w3 = np.random.randn(self.hiddenLayer2S, self.outputLayerS)
        self.B1 = np.random.randn(self.hiddenLayer1S)
        self.B2 = np.random.randn(self.hiddenLayer2S)
        
        self.B1 = np.random.randn(self.hiddenLayer1S)
        self.B2 = np.random.randn(self.hiddenLayer2S)
    
    def forward(self, x):
        self.z0 = x
        self.f0 = x#self.sigmoid(self.z0)
        self.z1 = np.dot(self.f0, self.w1)
        self.f1 = self.sigmoid(self.z1 + self.B1)
        self.z2 = np.dot(self.f1, self.w2)
        self.f2 = self.sigmoid(self.z2 + self.B2)
        self.z3 = np.dot(self.f2, self.w3)
        self.f3 = self.sigmoid(self.z3)
        return self.f3
    
    def sigmoid(self, z):
        return expit(z)
    
    def sigmoidP(self, x):
        #x = self.sigmoid(x)
        return x * (1.0 - x)
    
    def costFunc(self, res, tar):
        #tar = target the digit value
        #res = result of the forward NN
        yHat = self.forward(res)
        return np.sum((tar-yHat)**2)

    def costFuncPrime(self, exp, z, res):
        #z = image of digit in a single array
        
        lr = 1
        
        y = (exp - res)*2
        delta3 = y*self.sigmoidP(self.f3)
        dcdw3 = np.dot(self.f2.T,delta3)
        
        
        delta2 = np.dot(delta3,self.w3.T) * self.sigmoidP(self.f2)
        dcdw2 = np.dot(self.f1.T,delta2)
        
        delta1 = np.dot(delta2,self.w2.T) * self.sigmoidP(self.f1)
        dcdw1 = np.dot(self.f0.T,delta1)
        
        dw3 = (dcdw3)
        dw2 = (dcdw2)
        dw1 = (dcdw1)
        
        #db1 = np.sum(self.dcdb1, axis=0)
        #db2 = np.sum(self.dcdb2, axis=0)
        return dw3, dw2, dw1#, db1, db2
    
    def trainBias(self, z, exp):
        res=self.forward(z)
        #z = image of digit in a single array
        
        lr = 1
        
        y = (exp - res)*2
        delta3 = y*self.sigmoidP(self.f3)       
        
        delta2 = np.dot(delta3,self.w3.T) * self.sigmoidP(self.f2)
        
        delta1 = np.dot(delta2,self.w2.T) * self.sigmoidP(self.f1)
        
        self.B2 += np.sum(delta2, axis=0)
        self.B1 += np.sum(delta1, axis=0)
        
        return
    
    def trainNN(self, image, target):
        res=self.forward(image)
        dw3, dw2, dw1= self.costFuncPrime(target, image, res)
        return dw3, dw2, dw1
                
    def testNN(self, image, tar):
        res = self.forward(image)
        maxInd = np.argmax(res)
        y = np.zeros(10)
        y[tar]=1
        return maxInd, res, self.costFunc(image, y)

In [14]:
rec = neuralNetwork()
setSize = 100
learnRate = 0.001
y = np.zeros((setSize,10))

In [15]:
#Train the Weights only

for iteration in range(100):
    for i in range(500):
        a = digits.target[i*setSize:(i+1)*setSize]
        for j in range(100):
            y[j][a[j]]=1
        dw3, dw2, dw1= rec.trainNN(digits.data[i*setSize:(i+1)*setSize], y)
        rec.w1 = rec.w1 + dw1*learnRate
        rec.w2 = rec.w2 + dw2*learnRate
        rec.w3 = rec.w3 + dw3*learnRate

        #rec.B1 = rec.B1 + db1*learnRate
        #rec.B2 = rec.B2 + db2*learnRate
        y = np.zeros((setSize,10))
    if iteration%10==0:
        print(iteration)

#teach.train(digits.data[i*setSize:(i+1)*setSize], y)
    
print("Done")


0
10
20
30
40
50
60


KeyboardInterrupt: 

In [ ]:
#Trains Bias Only

for iteration in range(100):
    for i in range(500):
        a = digits.target[i*setSize:(i+1)*setSize]
        for j in range(100):
            y[j][a[j]]=1
        rec.trainBias(digits.data[i*setSize:(i+1)*setSize], y)
        y = np.zeros((setSize,10))
    if iteration%10==0:
        print(iteration)

#teach.train(digits.data[i*setSize:(i+1)*setSize], y)
    
print("Done")


In [ ]:
#Trains Both

for period in range(10):
    for iteration in range(100):
        for i in range(500):
            a = digits.target[i*setSize:(i+1)*setSize]
            for j in range(100):
                y[j][a[j]]=1
            dw3, dw2, dw1= rec.trainNN(digits.data[i*setSize:(i+1)*setSize], y)
            rec.w1 = rec.w1 + dw1*learnRate
            rec.w2 = rec.w2 + dw2*learnRate
            rec.w3 = rec.w3 + dw3*learnRate

            #rec.B1 = rec.B1 + db1*learnRate
            #rec.B2 = rec.B2 + db2*learnRate
            y = np.zeros((setSize,10))
    for iteration in range(100):
        for i in range(500):
            a = digits.target[i*setSize:(i+1)*setSize]
            for j in range(100):
                y[j][a[j]]=1
            rec.trainBias(digits.data[i*setSize:(i+1)*setSize], y)
            y = np.zeros((setSize,10))
    
    print(period)
    
print("Done")

In [ ]:
countTrue = 0
countAvgCost = 0
countLoop = 0
maxCost = 0

digitTest2 = 10
tempPer, res, cost = rec.testNN(digits.data[digitTest2], digits.target[digitTest2])

print("Obtained Result:")
print(tempPer)
print("\nPercentage Result:")
print(res)
print("\nActual Result:")
print(digits.target[digitTest2])
print("\nCost Result:")
print(cost)

for digitTest in range(50000,70000):
    countLoop += 1
    tempPer, res, cost = rec.testNN(digits.data[digitTest], digits.target[digitTest])
    countAvgCost += cost
    if tempPer==digits.target[digitTest]:
        countTrue += 1
    if maxCost < cost:
        maxCost = cost

print("\nPercentage Accurary")
print((countTrue/countLoop)*100)
print("\nAverage Cost")
print((countAvgCost/countLoop))
print("\nMax Cost")
print(maxCost)

In [ ]:
Obtained Result:
6

Percentage Result:
[  1.74355648e-03   8.04145056e-03   6.88176395e-02   9.84986907e-04
   1.17271835e-03   1.35756255e-03   9.73189897e-01   2.89744526e-03
   2.80046910e-04   4.29808939e-03]

Actual Result:
6

Cost Result:
0.00555348966948

Percentage Accurary
91.11

Average Cost
0.142146723662

Max Cost
2.03348259067

Obtained Result:
6

Percentage Result:
[  8.36008344e-05   3.62860282e-03   2.95500362e-02   1.66688210e-04
   2.94312361e-04   8.48171335e-05   9.97579246e-01   1.27587169e-03
   3.12756920e-05   8.35636573e-05]

Actual Result:
6

Cost Result:
0.000893995844916

Percentage Accurary
92.935

Average Cost
0.11107519811

Max Cost
2.01015877758

In [ ]:
a = np.random.randn(10)*10
b = np.ones(10)
print(a)
print(b)
print(a-b)

In [ ]:
for i in range(10,20):
    print(i)

In [ ]:
a = np.ones(10)
b = np.zeros((2,10))
print(36**(1/2))

5
6
